## Importing Necessary Library

In [ ]:
import seaborn as sns
import pandas as pd
import numpy as np
import torch
import gc
from torch import nn
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics
import torch.nn.functional as F
from copy import deepcopy
from pymoo.optimize import minimize
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from pymoo.visualization.scatter import Scatter
from mpl_toolkits.mplot3d import Axes3D
import shap
from captum.attr import IntegratedGradients
from shap import DeepExplainer, summary_plot
from sklearn.preprocessing import StandardScaler
from captum.attr import GradientShap

## Defining Datapath and Loading Data

In [ ]:
DATAPATH_Film = "FilmTrust"

Ratings = pd.read_csv(f'{DATAPATH_Film}/ratings.txt', sep=' ', names=['userId', 'itemId', 'rating']) ## Extracting the ratings from the dataset
Ratings.drop_duplicates(inplace=True)
Ratings = Ratings.groupby(['userId', 'itemId'], as_index=False)['rating'].mean() ## Grouping the ratings by matching with userID and itemID
print("Ratings shape in the FilmTrust Dataset:", Ratings.shape)

Interaction_Count = Ratings.groupby('userId')['itemId'].nunique().reset_index() ## Determinin the intrecation cound of users towards item
Interaction_Count.rename(columns={'itemId': 'A_Interaction_Count'}, inplace=True)  ## Adding colun of Interaction Count

Ratings = Ratings.merge(Interaction_Count, on='userId', how='left') ## Adding Interaction Count into the rating

Trust = pd.read_csv(f'{DATAPATH_Film}/trust.txt', sep=' ', names=['userId', 'userId_1', 'trust']) ## Loading Trust Fature given in the dataset
print("Trust shape in the FilmTrust Dataset:", Trust.shape)
print(Ratings.shape)

## Extracting More fatures using Aggregating provided infromation in the Dataset

In [ ]:
Trust_Agg_O = Trust.groupby('userId')['trust'].agg(['sum']).reset_index() ## Aggregating the trust of other users
Trust_Agg_O.columns = ['userId', 'A_Sum_Trust_UsersO']

Trust_Agg_S = Trust.groupby('userId_1')['trust'].agg(['sum']).reset_index() ## Aggregating the trust of sel
Trust_Agg_S.columns = ['userId', 'A_Sum_Trust_UsersS']

Ratings = pd.merge(Ratings, Trust_Agg_S, on='userId', how='left') ## Merging the aggregated feature infromation to the rating
Ratings = pd.merge(Ratings, Trust_Agg_O, on='userId', how='left')
print("Ratings shape after merging trust features:", Ratings.shape)

D_Ratings = Ratings.copy() ## Copying the ratings data

Users = D_Ratings['userId'].unique()
Users = np.random.permutation(Users)
Normal_Seg = 0.9
Num_Normal = int(np.round(len(Users) * Normal_Seg))
Users_Normal = Users[:Num_Normal]

D_Normal = D_Ratings[D_Ratings['userId'].isin(Users_Normal)]
print("Normal Users Shape:", D_Normal.shape)

## Segregating the Dataset into training and testing data

In [ ]:
Temp_train, Temp_valid = train_test_split(D_Normal, test_size = 0.2, train_size = 0.8, random_state=42, shuffle=True)
Temp_train['random_dstype'] = 'train'
Temp_valid['random_dstype'] = 'test'

D_Normal = pd.concat([Temp_train, Temp_valid], axis=0)
print(D_Normal['random_dstype'].value_counts())

In [ ]:
D_train = D_Normal[D_Normal['random_dstype'] == 'train'].copy()
D_valid = D_Normal[D_Normal['random_dstype'] == 'test'].copy()

assert D_train['userId'].isin(Users_Normal).all(), "Train set contains cold-start users!"
assert D_valid['userId'].isin(Users_Normal).all(), "Valid set contains cold-start users!"

print(D_train.shape)
print(D_valid.shape)

D_train['flag_train'] = 1
D_valid['flag_train'] = 0

D_train, D_valid = train_test_split(D_Normal, test_size = 0.1, train_size = 0.9, random_state=42, shuffle=True)
User_Features = D_train.columns[D_train.columns.str.startswith('A_')]

print("Final Train shape:", D_train.shape)
print("Final Valid shape :", D_valid.shape)

for feature in User_Features:
    D_train[f'A_na_{feature[2:]}'] = D_train[feature].isnull()
    D_train[feature].fillna(0, inplace=True)

for feature in User_Features:
    D_valid[f'A_na_{feature[2:]}'] = D_valid[feature].isnull()
    D_valid[feature].fillna(0, inplace=True)


print(D_train.head())
print("Final Train shape:", D_train.shape)
print("Final Valid shape :", D_valid.shape)


## Saving the Training and testin Data into CSV File

In [ ]:
D_train.to_csv("D_train_FilmTrust.csv", index=False)
D_valid.to_csv("D_valid_FilmTrust.csv", index=False)

print("Final Train shape:", D_train.shape)
print("Final Valid shape :", D_valid.shape)

## Embedded Feature-wise Linear Modulation (FiLM) DNN Model

In [ ]:
class EmbFilm(nn.Module):
    def __init__(self, N_Users, N_Items, Dense_Size, Hidden, Dropout_emb, Dropouts):
        super().__init__()
        self.Emb_User = nn.Embedding(N_Users + 1, 25)
        self.Emb_Item = nn.Embedding(N_Items + 1, 25)
        self.Hidden = Hidden
        self.Emb_Dropout = nn.Dropout(Dropout_emb)

        self.LinC1 = nn.Sequential(
            nn.Linear(Dense_Size, Hidden[1]),
            nn.ReLU(),
            nn.LayerNorm(Hidden[1]),
            nn.Dropout(Dropouts[1])
        )
        self.Emb_Projection = nn.Linear(50, Hidden[1])

        self.Film_Gamma1 = nn.Linear(2 * Hidden[1], Hidden[1])
        self.Film_Gamma2 = nn.Linear(2 * Hidden[1], Hidden[1])
        self.Film_Beta = nn.Linear(2 * Hidden[1], Hidden[1])

        self.LinC2 = nn.Sequential(
            nn.Linear(Hidden[1], Hidden[2]),
            nn.ReLU(),
            nn.LayerNorm(Hidden[2]),
            nn.Dropout(Dropouts[2])
        )
        
        self.LinC3 = nn.Linear(Hidden[2], 1)
        self.criterion = nn.MSELoss()

    def predict_fn(self, Side_np, device):
        Side_tensor = torch.tensor(Side_np, dtype=torch.float32).to(device)
        with torch.no_grad():
            Users = torch.zeros(Side_tensor.size(0), dtype=torch.long).to(device)
            Items = torch.zeros_like(Users)
            User_EmbV = self.Emb_User(Users)
            Item_EmbV = self.Emb_Item(Items)

            EmbV_Concat = torch.cat([User_EmbV, Item_EmbV], dim=1)
            EmbV_Proj = self.Emb_Projection(EmbV_Concat)

            Out1 = self.LinC1(Side_tensor)
            Cat_Out_UISFeat = torch.cat([EmbV_Proj, Out1], dim=1)

            Gamma1 = self.Film_Gamma1(Cat_Out_UISFeat)
            Gamma2 = self.Film_Gamma2(Cat_Out_UISFeat)
            Beta = self.Film_Beta(Cat_Out_UISFeat)

            Aggregated_Film = Gamma1 * EmbV_Proj + Gamma2 * Out1 + Beta
            Out2 = self.LinC2(Aggregated_Film)
            Out_Rat = self.LinC3(Out2)
            return torch.clamp(Out_Rat, 1.0, 5.0).cpu().numpy()

    def forward(self, Xb, Yb=None, shap_mode='kernel', use_shap=True):
        device = Xb.device
        Users = Xb[:, 0].long()
        Items = Xb[:, 1].long()
        SideFeat_info = Xb[:, 2:]

        User_EmbV = self.Emb_Dropout(self.Emb_User(Users))
        Item_EmbV = self.Emb_Dropout(self.Emb_Item(Items))
        EmbV_Concat = torch.cat([User_EmbV, Item_EmbV], dim=1)
        EmbV_Proj = self.Emb_Projection(EmbV_Concat)

        Out1 = self.LinC1(SideFeat_info)
        Cat_Out = torch.cat([EmbV_Proj, Out1], dim=1)
        Gamma1 = self.Film_Gamma1(Cat_Out)
        Gamma2 = self.Film_Gamma2(Cat_Out)
        Beta = self.Film_Beta(Cat_Out)

        Aggregated_Film = Gamma1 * EmbV_Proj + Gamma2 * Out1 + Beta
        Out2 = self.LinC2(Aggregated_Film)
        Out_Rat = self.LinC3(Out2)
        Out_Rat = torch.clamp(Out_Rat, 1.0, 5.0)

        if Yb is not None:
            Mse_Loss = self.criterion(Out_Rat, Yb)
            Mae_Loss = nn.L1Loss()(Out_Rat, Yb)
            Rmse_Loss = torch.sqrt(Mse_Loss)

            Shap_Values = torch.zeros_like(SideFeat_info)
            Penalty = torch.tensor(0.0, device = Out_Rat.device)

            if use_shap and shap_mode == 'kernel':
                try:
                    self.eval()
                    SideFeat_info_np = SideFeat_info.detach().cpu().numpy()
                    Background = SideFeat_info_np[np.random.choice(len(SideFeat_info_np), min(50, len(SideFeat_info_np)), replace=False)]

                    explainer = shap.KernelExplainer(lambda x: self.predict_fn(x, device), Background)
                    Shap_Values_np = explainer.shap_values(SideFeat_info_np, nsamples=100)
                    Shap_Values = torch.tensor(Shap_Values_np, dtype=torch.float32, device=Out_Rat.device)

                    Negative_Shap = torch.clamp(Shap_Values, max=0.0)
                    Negative_ShapA = Negative_Shap.abs()
                    Num_Negative_Shap = (Negative_Shap < 0).sum(dim=1).float()
                    Per_Sample_Penalty = Negative_ShapA.sum(dim=1) / (Num_Negative_Shap + 1e-6) / Shap_Values.shape[1]
                    Penalty = Per_Sample_Penalty.mean()
                    #print("Kernel SHAP Penalty:", Penalty.item())
                except Exception as e:
                    print("Kernel SHAP failed:", e)

            return Out_Rat, Rmse_Loss, Mae_Loss, Shap_Values, Penalty

        return Out_Rat
            

## For Preparing Model

In [ ]:
class Model_cfdata(Dataset):
    def __init__(self, D_X, D_Y, DenseCols):
        self.D_X = D_X
        self.D_Y = D_Y
        self.DenseCols = DenseCols

    def __len__(self):
        return len(self.D_X)

    def __getitem__(self, idx):
        x = self.D_X.iloc[idx].values.astype(np.float32)
        y = np.array([self.D_Y.iloc[idx]], dtype=np.float32)
        return torch.FloatTensor(x), torch.FloatTensor(y)

## Mapping UserID and ItemID indices for training and valid

In [ ]:
User2idx = {uid: idx for idx, uid in enumerate(D_train['userId'].unique())}
Item2idx = {iid: idx for idx, iid in enumerate(D_train['itemId'].unique())}

D_train['user_idx'] = D_train['userId'].map(User2idx)
D_train['item_idx'] = D_train['itemId'].map(Item2idx)

D_valid = D_valid[D_valid['userId'].isin(User2idx) & D_valid['itemId'].isin(Item2idx)].copy()
D_valid['user_idx'] = D_valid['userId'].map(User2idx)
D_valid['item_idx'] = D_valid['itemId'].map(Item2idx)

## Defining Relevant Features

In [ ]:
DenseCols = D_train.columns[D_train.columns.str.startswith('A_')].tolist()
InputCols = ['user_idx', 'item_idx'] + DenseCols

DenseCols = D_train.columns[D_train.columns.str.startswith('A_')].tolist()
InputCols = ['user_idx', 'item_idx'] + DenseCols

scaler = StandardScaler()
D_train[DenseCols] = scaler.fit_transform(D_train[DenseCols])
D_valid[DenseCols] = scaler.transform(D_valid[DenseCols])

## Converting training data into float32 numpy array

In [ ]:
X_train_np = D_train[InputCols].apply(pd.to_numeric, errors='coerce').fillna(0).astype(np.float32).values
y_train_np = D_train['rating'].astype(np.float32).values

X_val_np = D_valid[InputCols].apply(pd.to_numeric, errors='coerce').fillna(0).astype(np.float32).values
y_val_np = D_valid['rating'].astype(np.float32).values

X_train_Tens = torch.tensor(X_train_np)
y_train_Tens = torch.tensor(y_train_np)

X_val_Tens = torch.tensor(X_val_np)
y_val_Tens = torch.tensor(y_val_np)

## Defining Model Setup

In [ ]:
Model = EmbFilm(N_Users=D_train['user_idx'].nunique(), N_Items=D_train['item_idx'].nunique(),
           Dense_Size=len(DenseCols), Hidden=[20, 25, 10],
           Dropout_emb=0.05, Dropouts=[0.2, 0.3, 0.2])

num_weights = sum(p.numel() for p in Model.parameters())
weight_shape = (num_weights,)

In [ ]:
X_train = D_train[['user_idx', 'item_idx'] + DenseCols]
X_valid = D_valid[['user_idx', 'item_idx'] + DenseCols]
y_train = D_train['rating']
y_valid = D_valid['rating']
DL_train = DataLoader(Model_cfdata(X_train, y_train, DenseCols), batch_size=128, shuffle=True)
DL_valid = DataLoader(Model_cfdata(X_valid, y_valid, DenseCols), batch_size=128)

print(X_train.shape)
print(X_valid.shape)

## Function for training the Model

In [ ]:
class Learner:
    def __init__(self, Model, optimizer, device):
        self.Model = Model.to(device)
        self.optimizer = optimizer
        self.device = device
        self.Best_Rmse = float('inf')
        self.Best_Mae = float('inf')

        self.Loss_log = []
        self.Rmse_log = []
        self.Mae_log = []
        self.Penalty_log = []

    def compute_shap_penalty(self, Penalty, Lambda_Shap=1.0):
        if isinstance(Penalty, torch.Tensor):
            Mean_Val = Penalty.item()
        else:
            Mean_Val = float(Penalty)
        return Lambda_Shap * Penalty

    def fit(self, DL_train, DL_valid, n_epochs=5, Lambda_Shap=1.0):
        for epoch in range(n_epochs):
            self.Model.train()
            Total_Loss = 0
            preds_all, actual_all = [], []
            epoch_penalty = 0.0
            for Xb, Yb in DL_train:
                Xb, Yb = Xb.to(self.device), Yb.to(self.device)
                self.optimizer.zero_grad()
                    
                preds, Rmse_Loss, Mae_Loss, Shap_Values, Penalty = self.Model(Xb, Yb)  
                shap_penalty = self.compute_shap_penalty(Penalty, Lambda_Shap=Lambda_Shap)
                epoch_penalty += shap_penalty.item()

                Loss = Rmse_Loss + Mae_Loss + shap_penalty
                Loss = Loss/3.0
                Loss.backward()
                self.optimizer.step()

                Total_Loss += Loss.item()
                preds_all.extend(preds.detach().cpu().numpy())
                actual_all.extend(Yb.cpu().numpy())

            Rmse = np.sqrt(metrics.mean_squared_error(actual_all, preds_all))
            Mae = metrics.mean_absolute_error(actual_all, preds_all)
            self.Rmse_log.append(Rmse)
            self.Mae_log.append(Mae)
            self.Penalty_log.append(epoch_penalty)
            self.Loss_log.append(Total_Loss/len(DL_train))

            print(f"Epoch {epoch+1} - Train Loss: {Total_Loss/len(DL_train):.4f}, RMSE: {Rmse:.4f}, MAE: {Mae:.4f}, Penalty: {self.Penalty_log[-1]:.4f}")
            self.evaluate(DL_valid)

    def evaluate(self, DL_valid):
            self.Model.eval()
            preds_all, actual_all = [], []
            with torch.no_grad():
                for Xb, Yb in DL_valid:
                    Xb, Yb = Xb.to(self.device), Yb.to(self.device)
                    preds, _, _, _, _ = self.Model(Xb, Yb)  # Ignore SHAP during eval
                    preds_all.extend(preds.cpu().numpy())
                    actual_all.extend(Yb.cpu().numpy())
        
            D_valid['pred_rat'] = np.array(preds_all).flatten()
            Rmse = np.sqrt(metrics.mean_squared_error(D_valid['rating'], D_valid['pred_rat']))
            Mae = metrics.mean_absolute_error(D_valid['rating'], D_valid['pred_rat'])
            R2 = metrics.r2_score(D_valid['rating'], D_valid['pred_rat'])
            print(f"Validation RMSE: {Rmse:.4f}, MAE: {Mae:.4f}, R²: {R2:.4f}")


    def plot_logs(self):         
            min_len = min(len(self.Rmse_log), len(self.Mae_log), len(self.Penalty_log), len(self.Loss_log))
            epochs = list(range(1, min_len + 1))
            rmse_log = self.Rmse_log[:min_len]
            mae_log = self.Mae_log[:min_len]
            penalty_log = self.Penalty_log[:min_len]
            loss_log = self.Loss_log[:min_len]
        
            plt.figure(figsize=(15, 5))
        
            plt.subplot(1, 3, 1)
            plt.plot(epochs, loss_log, label="Total Loss")
            plt.xlabel("Epoch")
            plt.ylabel("Error")
            plt.title("Total Loss over Epochs")
            plt.legend()
         
            plt.subplot(1, 3, 2)
            plt.plot(epochs, rmse_log, label="RMSE")
            plt.plot(epochs, mae_log, label="MAE")
            plt.xlabel("Epoch")
            plt.ylabel("Error")
            plt.title("RMSE and MAE over Epochs")
            plt.legend()
            
            plt.subplot(1, 3, 3)
            plt.plot(epochs, penalty_log, color='purple')
            plt.xlabel("Epoch")
            plt.ylabel("SHAP Penalty")
            plt.title("Penalty from SHAP Constraint")
            
            plt.tight_layout()
            plt.show()    

In [ ]:
Learner_Model = Learner(Model, torch.optim.Adam(Model.parameters(), lr=5e-3, weight_decay=5e-4), device='cuda:0')
Learner_Model.fit(DL_train, DL_valid, n_epochs=1, Lambda_Shap=1.0)

In [ ]:
Learner_Model.plot_logs()